this notebook was an incomplete attempt to try HCNN with metadata as well (used previously in m02 variants)

TODO (out of scope): try this

In [49]:
# to do: remove batch_size related info

class CustomDataGeneratorForV03(torch.utils.data.Dataset):
    """
    Implementation notes: 
    you must have a column consisting of image names (with extension) as index of the dataframe (example: ['img0001.jpg', 'img0002.png', ...])
    each image name has to be unique across the entire dataset, not just its class

    This class is for PyTorch, unlike CustomDataGeneratorForV02, which was for TF. 
    """
    def __init__(self, img_labels_df, img_props_df, feature_cols, class_indices, target_size, batch_size, transforms_function=None):
        # attributes that were in flow_from_dataframe function used previously in CustomDataGeneratorForV02. Added here for compatibility with other functions like trainModel(), etc
        self.img_labels_df = img_labels_df
        self.filenames = list(img_labels_df['filepath'])
        self.labels = list(img_labels_df['label']) # this is actually not found in flow_from_dataframe's __dict__, but is a synonym of "classes" attribute found in the __dict__
        self.samples = len(img_labels_df) # number of rows of the df
        self.class_indices = class_indices
        self.target_size = target_size
        self.batch_size = batch_size

        self.imgp_df = img_props_df # imgp_df means "image props dataframe"
        self.feature_cols = feature_cols
        self.transforms_function = transforms_function


    def __len__(self):
        return self.samples // self.batch_size
    
    def __getitem__(self, index):
        # source: https://stackoverflow.com/questions/65231299/load-csv-and-image-dataset-in-pytorch

        filepath = self.filenames[index]
        img = Image.open(filepath)
        if self.transforms_function is not None:
            img = self.transforms_function(img)
        
        img_props = list(self.imgp_df.loc[os.path.basename(filepath), self.feature_cols])
        img_props = torch.tensor(img_props)

        x = [img, img_props]

        label = self.labels[index]
        y = self.class_indices[label] # mapping class name to its integer counterpart
        y = torch.tensor(y, dtype=torch.int8)

        return x, y

    # to do: delete these commented code if __getitem__() works
    # def __getitem__(self, index):
    #     # "index" will be a list of indices in case PyTorch's batch_sampler class is passed in PyTorch's Dataloader (code shown in a later cell)
    #     # source: https://discuss.pytorch.org/t/force-dataloader-to-fetch-batched-index-from-custom-batch-sampler/86656/2#:~:text=If%20you%20run%20it%2C%20you%E2%80%99ll%20see%20that%20the%20index%20inside%20the%20Dataset.__getitem__%20will%20contain%2010%20indices%2C%20which%20can%20be%20used%20to%20slice%20the%20data%20directly.
    #     img_labels_df_batch = self.img_labels_df.iloc[index]
    #     filepaths_batch = list(img_labels_df_batch['filepath'])
    #     labels_batch = list(img_labels_df_batch['label'])
    #     y_batch = [self.class_indices[label] for label in labels_batch] # mapping class names to their integer counterparts

    #     img_batch = [Image(img_path) for img_path in filepaths_batch]


    #     # to do: delete this part if other __getitem()__ doesn't work (and make its logic pytorch related):
    #     # Getting the corresponding df rows for each image in the batch
    #     filepaths_batch = self.filenames[index * self.batch_size : (index + 1) * self.batch_size] # note that self.filenames/batch_index/batch_size are all attributes initialized upon calling "super().__init__(*args, **kwargs)"
    #     image_filenames_batch = [os.path.basename(filepath) for filepath in filepaths_batch]
    #     img_props_batch = self.imgp_df.loc[image_filenames_batch, self.feature_cols]
    #     x_batch = [img_batch, img_props_batch]
    #     return x_batch, y_batch
    


In [50]:
import torchvision
import torchvision.transforms as T

feature_cols_for_v02 = [
    'dominant_color_1',
    'color_to_image_ratio_1',
    'dominant_color_2',
    'color_to_image_ratio_2',
    'dominant_color_3',
    'color_to_image_ratio_3',
    'dominant_color_4',
    'color_to_image_ratio_4',
    'dominant_color_5',
    'color_to_image_ratio_5'
 ]
target_size = (306, 306)
transforms_function = T.Compose(
    [
        T.Resize(target_size, interpolation=torchvision.transforms.InterpolationMode.NEAREST),
        T.ToTensor()
    ]
)
ddgen = CustomDataGeneratorForV03(train_data, df, feature_cols_for_v02, classNameToNum, target_size, 32, transforms_function)